In [1]:
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
import os, shutil
data_list = []
folder = '/content/drive/MyDrive/Study/ML/dataset'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    with open(file_path, 'r') as f:
        text = f.read()
        data_list.append(text)
data = " ".join(data_list)

In [3]:
def tokenize_words(input):
    # lowercase everything to standardize it
    input = input.lower()

    # instantiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # if the created token isn't in the stop words, make it part of "filtered"
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

In [4]:
# preprocess the input data, make tokens
processed_inputs = tokenize_words(data)

In [5]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [6]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 54049
Total vocab: 37


In [7]:
seq_length = 100
x_data = []
y_data = []

In [8]:
# loop through inputs, start at the beginning and go until we hit
# the final character we can create a sequence out of
for i in range(0, input_len - seq_length, 1):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = processed_inputs[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = processed_inputs[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

In [9]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)


Total Patterns: 53949


In [10]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [11]:
y = np_utils.to_categorical(y_data)


In [12]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [20]:
model.fit(X, y, epochs=20, batch_size=256)


Epoch 1/20
211/211 [==============================] - 11s 54ms/step - loss: 2.6475
Epoch 2/20
211/211 [==============================] - 11s 54ms/step - loss: 2.5960
Epoch 3/20
211/211 [==============================] - 11s 54ms/step - loss: 2.5349
Epoch 4/20
211/211 [==============================] - 11s 54ms/step - loss: 2.4777
Epoch 5/20
211/211 [==============================] - 11s 54ms/step - loss: 2.4302
Epoch 6/20
211/211 [==============================] - 11s 54ms/step - loss: 2.3878
Epoch 7/20
211/211 [==============================] - 11s 54ms/step - loss: 2.3496
Epoch 8/20
211/211 [==============================] - 11s 54ms/step - loss: 2.3143
Epoch 9/20
211/211 [==============================] - 11s 54ms/step - loss: 2.2815
Epoch 10/20
211/211 [==============================] - 11s 54ms/step - loss: 2.2530
Epoch 11/20
211/211 [==============================] - 11s 54ms/step - loss: 2.2242
Epoch 12/20
211/211 [==============================] - 11s 53ms/step - loss: 2.1987
E

In [21]:
num_to_char = dict((i, c) for i, c in enumerate(chars))


In [27]:
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" friendship entire complete used showed yore little interest pupil brussels hold maintenance little i "


In [28]:
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = num_to_char[index]

    sys.stdout.write(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]

odeers see shall see shall see shall see shall see shall see shall see shall see shall see shall see